In [1]:
import torch
import shap
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models, transforms
from PIL import Image
import os

In [ ]:

# # 1. Load ResNet50 model
# model = models.resnet50(pretrained=False)
# model.fc = torch.nn.Linear(model.fc.in_features, 4)  # 4 classes
# model.load_state_dict(torch.load("brain_tumor_model.pth", map_location=torch.device('cpu')))
# model.eval()

# # 2. Class labels
# class_names = ['Glioma','Meningioma','No Tumor', 'Pituitary']

# # 3. Image preprocessing (ImageNet-style)
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
#                          std=[0.229, 0.224, 0.225])
# ])


/home/tanuj/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tanuj/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:

# 4. Load and preprocess the MRI image
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0)  # Add batch dim
    return image, image_tensor


In [4]:

# 5. SHAP uses background data – select 5 random background images
def get_background_images(folder, num=5):
    images = []
    for fname in os.listdir(folder)[:num]:
        path = os.path.join(folder, fname)
        img = Image.open(path).convert('RGB')
        images.append(transform(img))
    return torch.stack(images)

# 6. Run SHAP explanation
def explain_with_shap(image_path, background_folder):
    pil_img, input_tensor = load_image(image_path)

    # Get background data
    background = get_background_images(background_folder)

    # Use GradientExplainer for PyTorch models
    explainer = shap.GradientExplainer(model, background)
    shap_values, indexes = explainer.shap_values(input_tensor)

    # Get prediction
    with torch.no_grad():
        output = model(input_tensor)
        probs = torch.nn.functional
